Early regression attempts not used in final modeling process but kept in case I decided to return to regression. 


In [50]:
from sklearn.ensemble import  AdaBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [51]:
#Bring in the data

nba = pd.read_csv('./data/nba_analysis_data.csv')
nba.dropna(inplace = True)

In [52]:
#seperate test from train

nba_train = nba[nba.test == 0]
nba_test = nba[nba.test ==1]

In [53]:
nba_train = nba_train[nba_train.lastaway_cover == 1]
nba_test = nba_test[nba_test.lastaway_cover == 1]

In [54]:
X_train = nba_train.drop(columns = ['cover','home_win_margin', 
                        'date', 'dataset','line_cv',
                       'home_starter5', 'teams', 'away_team',
                        'away_starter2', 'away_starter3',
                       'away_starter4', 'away_starter5',
                       'ref_1', 'ref_3', 'crew_referees',
                      'away_pace', 'away_spread',
                        'away_line_cv',  'away_cover',
                       'home_payout', 'away_payout'])

y_train = nba_train['home_win_margin']

In [55]:
X_test= nba_test.drop(columns = ['cover','home_win_margin', 
                        'date', 'dataset','line_cv', 
                       'home_starter5', 'teams', 'away_team',
                       'away_starter2', 'away_starter3',
                       'away_starter4', 'away_starter5',
                       'ref_1', 'ref_3', 'crew_referees',
                      'away_pace', 'away_spread',
                        'away_line_cv',  'away_cover',
                       'home_payout', 'away_payout'])

y_test = nba_test.home_win_margin

In [56]:
print(X_train.shape)
print(X_test.shape)

(1259, 154)
(248, 154)


In [57]:

# #create interactions and squared terms
# poly = PolynomialFeatures(degree = 2, interaction_only= True)
# X_poly = poly.fit_transform(X_train)
# X_poly_test = poly.transform(X_test)
# X_poly = pd.DataFrame(X_poly, 
#                       columns = poly.get_feature_names(X_train.columns))
# X_poly_test = pd.DataFrame(X_poly_test, 
#                            columns = poly.get_feature_names(X_test.columns))


In [58]:
#scaling data to use in various other methods
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [60]:
from sklearn.decomposition import PCA


In [61]:
pc = PCA(n_components=70)
X_train_pc = pc.fit_transform(X_train_scaled)
X_test_pc = pc.transform(X_test_scaled)



In [62]:

var_exp = pc.explained_variance_ratio_
print('Explained variance:            ', var_exp)

cum_var_exp = np.cumsum(var_exp)
print('Cumulative explained variance: ', cum_var_exp)

Explained variance:             [0.09039406 0.0736711  0.05754853 0.04814736 0.04278244 0.03288969
 0.03066873 0.02674754 0.02611886 0.02472816 0.02274306 0.02211918
 0.02061283 0.01938838 0.0182176  0.01755207 0.01572987 0.0154184
 0.01446179 0.01362774 0.01296714 0.01252083 0.0119348  0.01159095
 0.01133983 0.01073093 0.01051369 0.01030307 0.01012268 0.00952101
 0.00922886 0.0091163  0.00895066 0.00852385 0.00844372 0.00790249
 0.00760893 0.00739523 0.00718977 0.00705772 0.00694309 0.00671362
 0.00661543 0.00646414 0.00636841 0.00626555 0.00592569 0.00564248
 0.00557051 0.00515219 0.00500183 0.00479114 0.00459808 0.00438497
 0.00431044 0.00419045 0.00393646 0.00364252 0.0035196  0.00344396
 0.00330696 0.0032047  0.00317435 0.00280706 0.00275416 0.00266247
 0.00260757 0.00253026 0.00251001 0.002445  ]
Cumulative explained variance:  [0.09039406 0.16406516 0.22161369 0.26976105 0.31254349 0.34543318
 0.37610191 0.40284945 0.42896831 0.45369647 0.47643953 0.49855871
 0.51917155 0.538559

### Linear Regression

In [63]:
ols = LinearRegression()
ols.fit(X_train_pc, y_train)
ols.score(X_train_pc, y_train)

0.24110687006274645

In [64]:
ols.score(X_test_pc, y_test)

0.13922476093672065

### Boosting

In [65]:
ad = AdaBoostRegressor()
ad_params = { 'n_estimators'      : [300],
             'learning_rate'         : [ .01 ],
             
            }

In [66]:
gs = GridSearchCV(ad, param_grid= ad_params)
gs.fit(X_train_pc, y_train)
print(gs.best_params_)
print(gs.best_score_)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'learning_rate': 0.01, 'n_estimators': 300}
0.06284444445874796


In [67]:
gs.score(X_test_pc,y_test)

0.06177627878216841

In [68]:
y_hat_ad = gs.predict(X_test_pc)

In [69]:
rr = RidgeCV(alphas = [70000, 90000, 80000])


In [70]:
rr.fit(X_train_pc, y_train);

In [71]:
rr.score(X_train_pc, y_train)

0.042765657254360545

In [72]:
rr.score(X_test_pc, y_test)

0.0340990242040754

In [73]:
y_hat_rr = rr.predict(X_test_pc)

In [74]:
rr.alpha_

70000

### Support Vector Machine

In [75]:
svm = SVR()
svm_params = {'kernel' : ['rbf'],
              'C'      : [.1, .02, .05]}

In [76]:
gs = GridSearchCV(svm, param_grid= svm_params)
gs.fit(X_train_pc, y_train)
print(gs.best_params_)
print(gs.best_score_)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/ba

{'C': 0.1, 'kernel': 'rbf'}
0.0008750757649268159


In [77]:
gs.score(X_test_pc, y_test)

-0.013926336114587823

In [78]:
y_hat_svm = gs.predict(X_test_pc)


### Neural Network

Going to fit a neural network with the X features.   I am not concerned about inference.  Accurate prediction is all that matters.  So am going to set up an neural network as one of the final predictors of Home Win Margin.

In [79]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [80]:
model = Sequential()
model.add(Dense(100, 
                activation = 'relu',
                input_dim = X_train_pc.shape[1]))
#model.add(Dense(18, activation = 'relu'))

model.add(Dense(1, activation = None)) #output layer 

In [81]:
model.compile(loss = 'mean_squared_error', optimizer='adam' )
model.fit(X_train_pc, 
          y_train, epochs =18, 
          batch_size= 64, 
          validation_data=(X_test_pc, y_test))

Train on 1259 samples, validate on 248 samples
Epoch 1/18
1259/1259 [==============================] - 0s 163us/step - loss: 172.7821 - val_loss: 174.2930
Epoch 2/18
1259/1259 [==============================] - 0s 20us/step - loss: 162.8499 - val_loss: 168.5917
Epoch 3/18
1259/1259 [==============================] - 0s 23us/step - loss: 154.9201 - val_loss: 163.7921
Epoch 4/18
1259/1259 [==============================] - 0s 27us/step - loss: 147.8382 - val_loss: 160.5076
Epoch 5/18
1259/1259 [==============================] - 0s 24us/step - loss: 142.0056 - val_loss: 157.7031
Epoch 6/18
1259/1259 [==============================] - 0s 25us/step - loss: 137.2153 - val_loss: 156.1869
Epoch 7/18
1259/1259 [==============================] - 0s 21us/step - loss: 133.0597 - val_loss: 155.1391
Epoch 8/18
1259/1259 [==============================] - 0s 24us/step - loss: 129.7323 - val_loss: 155.1226
Epoch 9/18
1259/1259 [==============================] - 0s 20us/step - loss: 126.8959 - val_loss

In [82]:
y_hat_nn = model.predict(X_test_pc)
predictions = pd.DataFrame(y_hat_nn, columns= ['y_hat_nn'])
predictions

,y_hat_nn
0,10.365058
1,5.088275
2,6.555200
3,1.098100
4,9.082647
5,-8.966017
6,-5.202365
7,-0.410180
8,-0.270109
9,-3.539596


In [83]:
predictions['y_hat_rr'] = y_hat_rr
predictions['y_hat_svm'] = y_hat_svm
predictions['y_hat_ad'] = y_hat_ad


In [84]:
predictions.head(10)

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad
0,10.365058,4.009576,4.072298,2.269360
1,5.088275,2.308346,3.880794,-0.225728
2,6.555200,2.843860,3.908377,1.647059
3,1.098100,2.504836,3.765078,0.776350
4,9.082647,3.540969,4.210789,4.891117
5,-8.966017,1.828173,3.665507,-2.310789
6,-5.202365,2.221624,3.559926,-0.346939
7,-0.410180,2.061582,3.731947,0.147806
8,-0.270109,2.969503,3.992380,0.784375
9,-3.539596,2.628776,3.805184,0.436538


In [85]:
y_s = pd.DataFrame(y_test)
y_s.reset_index(inplace = True)
predictions['y_test'] = y_s['home_win_margin']
predictions.head()

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test
0,10.365058,4.009576,4.072298,2.269360,-14.0
1,5.088275,2.308346,3.880794,-0.225728,-15.0
2,6.555200,2.843860,3.908377,1.647059,-1.0
3,1.098100,2.504836,3.765078,0.776350,23.0
4,9.082647,3.540969,4.210789,4.891117,18.0


In [86]:
X_test.reset_index(inplace = True)
predictions['spread'] = X_test['spread']
predictions

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread
0,10.365058,4.009576,4.072298,2.269360,-14.0,-10.0
1,5.088275,2.308346,3.880794,-0.225728,-15.0,3.5
2,6.555200,2.843860,3.908377,1.647059,-1.0,-3.0
3,1.098100,2.504836,3.765078,0.776350,23.0,0.0
4,9.082647,3.540969,4.210789,4.891117,18.0,-7.5
5,-8.966017,1.828173,3.665507,-2.310789,-1.0,8.5
6,-5.202365,2.221624,3.559926,-0.346939,-10.0,4.5
7,-0.410180,2.061582,3.731947,0.147806,-3.0,2.0
8,-0.270109,2.969503,3.992380,0.784375,-7.0,1.0
9,-3.539596,2.628776,3.805184,0.436538,-5.0,1.5


In [87]:
predictions['y_hat_ave'] = (predictions.y_hat_nn + 
                           
                            predictions.y_hat_rr + 
                            predictions.y_hat_nn)/3

In [88]:
predictions.head(20)

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread,y_hat_ave
0,10.365058,4.009576,4.072298,2.269360,-14.0,-10.0,8.246564
1,5.088275,2.308346,3.880794,-0.225728,-15.0,3.5,4.161632
2,6.555200,2.843860,3.908377,1.647059,-1.0,-3.0,5.318086
3,1.098100,2.504836,3.765078,0.776350,23.0,0.0,1.567012
4,9.082647,3.540969,4.210789,4.891117,18.0,-7.5,7.235421
5,-8.966017,1.828173,3.665507,-2.310789,-1.0,8.5,-5.367953
6,-5.202365,2.221624,3.559926,-0.346939,-10.0,4.5,-2.727702
7,-0.410180,2.061582,3.731947,0.147806,-3.0,2.0,0.413741
8,-0.270109,2.969503,3.992380,0.784375,-7.0,1.0,0.809762
9,-3.539596,2.628776,3.805184,0.436538,-5.0,1.5,-1.483472


In [89]:
predictions['predict_svm'] =  (predictions.y_hat_svm + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [90]:
predictions['predict_ad'] =  (predictions.y_hat_ad + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [91]:
predictions['predict_rr'] =  (predictions.y_hat_rr + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [92]:
predictions['predict_nn'] =  (predictions.y_hat_nn + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [93]:
predictions['actual'] = (predictions.y_test + predictions.spread).map(lambda x: 0 if x< 0 else 1)

In [94]:
predictions['predict'] = (predictions['predict_svm']+ 
    predictions['predict_rr'] + predictions['predict_nn'])

In [95]:
predictions.head()

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread,y_hat_ave,predict_svm,predict_ad,predict_rr,predict_nn,actual,predict
0,10.365058,4.009576,4.072298,2.269360,-14.0,-10.0,8.246564,0,0,0,1,0,1
1,5.088275,2.308346,3.880794,-0.225728,-15.0,3.5,4.161632,1,1,1,1,0,3
2,6.555200,2.843860,3.908377,1.647059,-1.0,-3.0,5.318086,1,0,0,1,0,2
3,1.098100,2.504836,3.765078,0.776350,23.0,0.0,1.567012,1,1,1,1,1,3
4,9.082647,3.540969,4.210789,4.891117,18.0,-7.5,7.235421,0,0,0,1,1,1


In [96]:
print(classification_report(predictions.actual, predictions.predict_rr))

              precision    recall  f1-score   support

           0       0.52      0.51      0.52       129
           1       0.48      0.50      0.49       119

   micro avg       0.50      0.50      0.50       248
   macro avg       0.50      0.50      0.50       248
weighted avg       0.50      0.50      0.50       248



In [97]:
confusion_matrix(predictions.actual, predictions.predict_svm)

array([[56, 73],
       [54, 65]])

In [98]:
predictions.predict.value_counts(normalize = True)

3    0.370968
1    0.270161
0    0.205645
2    0.153226
Name: predict, dtype: float64

In [99]:
predictions.groupby(['predict'])['actual'].mean()

predict
0    0.431373
1    0.522388
2    0.500000
3    0.467391
Name: actual, dtype: float64

In [100]:
predictions[predictions.predict == 0].away_payout.sum()

AttributeError: 'DataFrame' object has no attribute 'away_payout'